In [1]:
using CSV, DataFrames

In [2]:
forest_fires = CSV.read("./data/forestfires.csv", DataFrame)

Row,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
,Int64,Int64,String3,String3,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Float64
1,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
2,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
3,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
4,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
5,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0
6,8,6,aug,sun,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,0.0
7,8,6,aug,mon,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,0.0
8,8,6,aug,mon,91.5,145.4,608.2,10.7,8.0,86,2.2,0.0,0.0
9,8,6,sep,tue,91.0,129.5,692.6,7.0,13.1,63,5.4,0.0,0.0


In [3]:
using CategoricalArrays

function load_dataset(Y_name)
    # Loading and preprocessing
    forest_fires = CSV.read("./data/forestfires.csv", DataFrame)
    forest_fires.month = levelcode.(CategoricalArray(forest_fires.month))
    forest_fires.day = levelcode.(CategoricalArray(forest_fires.day))
    forest_fires = mapcols(col -> Float32.(col), forest_fires)

    # Getting X and Y
    X = select(forest_fires, Not(Y_name))
    Y = select(forest_fires, Y_name)
    return Matrix(X)', Matrix(Y)'
end

load_dataset (generic function with 1 method)

In [4]:
X, Y = load_dataset(:rain)

(Float32[7.0 7.0 … 1.0 6.0; 5.0 4.0 … 4.0 3.0; … ; 6.7 0.9 … 4.0 4.5; 0.0 0.0 … 0.0 0.0], Float32[0.0 0.0 … 0.0 0.0])

In [6]:
using Mlyn

X_train, Y_train, X_test, Y_test = train_test_split(X, Y, 0.8)

(Float32[6.0 4.0 … 3.0 2.0; 3.0 4.0 … 4.0 2.0; … ; 2.7 4.5 … 4.5 0.9; 0.0 88.49 … 0.0 6.84], Float32[0.0 0.0 … 0.0 0.0], Float32[3.0 1.0 … 1.0 3.0; 5.0 5.0 … 4.0 4.0; … ; 2.7 2.2 … 4.9 1.8; 2.18 7.3 … 9.71 0.0], Float32[0.0 0.0 … 0.0 0.0])

In [7]:
using Mlyn

model = create_model(regression, [
    BlueprintLinear(12, 5, method_xavier_normal),
    BlueprintActivation(method_sigmoid),
    BlueprintLinear(5, 3, method_xavier_normal),
    BlueprintActivation(method_sigmoid),
    BlueprintLinear(3, 1, method_xavier_normal),
    BlueprintActivation(method_sigmoid)
])

optimizer = OptimizerSGD(learning_rate=0.01)

OptimizerSGD(0.01f0)

In [8]:
train!(model, optimizer, 100, X_train, Y_train, X_test, Y_test; batch_size=100)

[EPOCH 1] Test loss = 0.09115090128485129; Learning loss = 0.36622094991460613
[EPOCH 2] Test loss = 0.08996068890482693; Learning loss = 0.12789663877393945
[EPOCH 3] Test loss = 0.08879031412510979; Learning loss = 0.12616387326311368
[EPOCH 4] Test loss = 0.08773026700433703; Learning loss = 0.13160330548186927
[EPOCH 5] Test loss = 0.08660407769125822; Learning loss = 0.12386653155873917
[EPOCH 6] Test loss = 0.08549004086443723; Learning loss = 0.12319099666359097
[EPOCH 7] Test loss = 0.084395124276685; Learning loss = 0.1217722164031519
[EPOCH 8] Test loss = 0.08331770844748844; Learning loss = 0.12130304122333724
[EPOCH 9] Test loss = 0.08249443308389146; Learning loss = 0.359613902919245
[EPOCH 10] Test loss = 0.08148651740749703; Learning loss = 0.12060151198609792
[EPOCH 11] Test loss = 0.08046633044511818; Learning loss = 0.11742473502247211
[EPOCH 12] Test loss = 0.07945410223579516; Learning loss = 0.11719448224807334
[EPOCH 13] Test loss = 0.07849522906683117; Learning l

100-element Vector{Any}:
 (0.09115090128485129, 0.36622094991460613)
 (0.08996068890482693, 0.12789663877393945)
 (0.08879031412510979, 0.12616387326311368)
 (0.08773026700433703, 0.13160330548186927)
 (0.08660407769125822, 0.12386653155873917)
 (0.08549004086443723, 0.12319099666359097)
 (0.084395124276685, 0.1217722164031519)
 (0.08331770844748844, 0.12130304122333724)
 (0.08249443308389146, 0.359613902919245)
 (0.08148651740749703, 0.12060151198609792)
 (0.08046633044511818, 0.11742473502247211)
 (0.07945410223579516, 0.11719448224807334)
 (0.07849522906683117, 0.11774815231376226)
 ⋮
 (0.03637318736275109, 0.0752076774464061)
 (0.03608064236305613, 0.07513495787224446)
 (0.03580952652385509, 0.0778525258926023)
 (0.03553895648776398, 0.0758933483257766)
 (0.035257721762857276, 0.0743808986073061)
 (0.034981948681045275, 0.07401829069941157)
 (0.0347121853716836, 0.07345349393189213)
 (0.03444394640930965, 0.07349698376664557)
 (0.03417969488499335, 0.07316402964693883)
 (0.03391813

In [14]:
X, Y = load_dataset(:area)

X_train, Y_train, X_test, Y_test = train_test_split(X, Y, 0.8)

area_model = create_model(regression, [
    BlueprintLinear(12, 5, method_xavier_normal),
    BlueprintActivation(method_sigmoid),
    BlueprintLinear(5, 3, method_xavier_normal),
    BlueprintActivation(method_sigmoid),
    BlueprintLinear(3, 1, method_xavier_normal),
    BlueprintActivation(method_sigmoid)
])

optimizer = OptimizerSGD(learning_rate=0.05)

train!(area_model, optimizer, 100, X_train, Y_train, X_test, Y_test; batch_size=1000)

[EPOCH 1] Test loss = 2898.7261576139977; Learning loss = 1898.3024060599212
[EPOCH 2] Test loss = 2897.570217054804; Learning loss = 1897.4713907880941
[EPOCH 3] Test loss = 2896.2462608800165; Learning loss = 1896.6044309261026
[EPOCH 4] Test loss = 2895.266866953088; Learning loss = 1895.7387527228016
[EPOCH 5] Test loss = 2894.4463311011523; Learning loss = 1895.0139316121947
[EPOCH 6] Test loss = 2893.7832973876884; Learning loss = 1894.4073170505703
[EPOCH 7] Test loss = 2893.2564349492354; Learning loss = 1893.9175424277655
[EPOCH 8] Test loss = 2892.8390385339912; Learning loss = 1893.5285897237595
[EPOCH 9] Test loss = 2892.506611157282; Learning loss = 1893.2205808570004
[EPOCH 10] Test loss = 2892.239288822548; Learning loss = 1892.9753447674416
[EPOCH 11] Test loss = 2892.02182659023; Learning loss = 1892.7781756555844
[EPOCH 12] Test loss = 2891.8427875554125; Learning loss = 1892.6178013993924
[EPOCH 13] Test loss = 2891.6936448340207; Learning loss = 1892.485772762981
[E

100-element Vector{Any}:
 (2898.7261576139977, 1898.3024060599212)
 (2897.570217054804, 1897.4713907880941)
 (2896.2462608800165, 1896.6044309261026)
 (2895.266866953088, 1895.7387527228016)
 (2894.4463311011523, 1895.0139316121947)
 (2893.7832973876884, 1894.4073170505703)
 (2893.2564349492354, 1893.9175424277655)
 (2892.8390385339912, 1893.5285897237595)
 (2892.506611157282, 1893.2205808570004)
 (2892.239288822548, 1892.9753447674416)
 (2892.02182659023, 1892.7781756555844)
 (2891.8427875554125, 1892.6178013993924)
 (2891.6936448340207, 1892.485772762981)
 ⋮
 (2890.429645322202, 1891.4445457272852)
 (2890.427918150969, 1891.4432374772891)
 (2890.4262324787956, 1891.4419610189868)
 (2890.4245868700705, 1891.440715208655)
 (2890.4229798858455, 1891.4394989866325)
 (2890.4214102566516, 1891.4383112913285)
 (2890.4198767067674, 1891.4371511850334)
 (2890.4183780236376, 1891.4360177266901)
 (2890.4169130486425, 1891.4349100207378)
 (2890.4154806922347, 1891.433827212238)
 (2890.4140798623